In [1]:
import torch

# Set PyTorch to use GPU 1
device = torch.device("cuda:2")
torch.cuda.set_device(device)

# Allocate a tensor to initialize GPU 1
x = torch.rand(1000, 1000, device=device)

# Verify the GPU being used
print(f"Using GPU: {torch.cuda.get_device_name(device.index)}")
print(f"Memory Allocated on GPU 1: {torch.cuda.memory_allocated(device.index) / 1024 ** 2:.2f} MB")
print(f"Memory Reserved on GPU 1: {torch.cuda.memory_reserved(device.index) / 1024 ** 2:.2f} MB")

Using GPU: Tesla V100-DGXS-32GB
Memory Allocated on GPU 1: 3.81 MB
Memory Reserved on GPU 1: 20.00 MB


In [2]:
import json
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomResizedCrop, ColorJitter, RandomRotation
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from timm import create_model

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define Image Transformations with more augmentation
image_transform = Compose([
    RandomHorizontalFlip(),
    RandomResizedCrop(224),
    RandomRotation(15),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

# Load XLM-RoBERTa tokenizer
xlmr_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Dataset class
class MultimodalDataset(Dataset):
    def __init__(self, data, image_transform, tokenizer):
        self.data = data
        self.image_transform = image_transform
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        record = self.data[idx]
        image_path = record["img"]
        image = Image.open(image_path).convert("RGB")
        image = self.image_transform(image)

        text = record["text"]
        text_tokens = self.tokenizer(
            text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=128
        )

        label = torch.tensor(record["label"], dtype=torch.float)
        return image, text_tokens["input_ids"].squeeze(0), text_tokens["attention_mask"].squeeze(0), label

# Improved CLIP-Like Fusion Module with more layers
class ImprovedCLIPFusion(nn.Module):
    def __init__(self, image_dim, text_dim, fusion_dim):
        super(ImprovedCLIPFusion, self).__init__()
        self.image_proj = nn.Linear(image_dim, fusion_dim)
        self.text_proj = nn.Linear(text_dim, fusion_dim)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(fusion_dim, fusion_dim)
        self.fc2 = nn.Linear(fusion_dim, fusion_dim)
        self.fc3 = nn.Linear(fusion_dim, 1)

    def forward(self, image_features, text_features):
        image_proj = self.activation(self.image_proj(image_features))
        text_proj = self.activation(self.text_proj(text_features))
        fused_features = torch.tanh(image_proj + text_proj)
        fused_features = self.dropout(fused_features)
        fused_features = self.activation(self.fc1(fused_features))
        fused_features = self.activation(self.fc2(fused_features))
        output = self.fc3(fused_features)
        return output

# Define the Complete Multimodal Model
class MultimodalClassifier(nn.Module):
    def __init__(self, swin_model_name, xlmr_model_name, fusion_dim):
        super(MultimodalClassifier, self).__init__()
        self.swin_model = create_model(swin_model_name, pretrained=True, num_classes=0)
        self.xlmr_model = XLMRobertaModel.from_pretrained(xlmr_model_name)
        self.clip_fusion = ImprovedCLIPFusion(
            image_dim=self.swin_model.num_features, 
            text_dim=self.xlmr_model.config.hidden_size, 
            fusion_dim=fusion_dim
        )

    def forward(self, images, texts, attention_masks):
        image_features = self.swin_model(images)
        text_features = self.xlmr_model(input_ids=texts, attention_mask=attention_masks).last_hidden_state[:, 0, :]  # Use [CLS] token
        output = self.clip_fusion(image_features, text_features)
        return output

# Function to load datasets
def load_dataset(file_path, is_jsonl=False):
    data = []
    with open(file_path, "r") as f:
        if is_jsonl:
            for line in f:
                try:
                    record = json.loads(line.strip())
                    data.append(record)
                except json.JSONDecodeError:
                    continue
        else:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                print(f"Error decoding JSON file: {file_path}")
    return data

# Training and Fine-Tuning Function
def train_and_fine_tune_model(model, train_loader, fine_tune_loader, criterion, optimizer, scheduler, device, train_epochs, fine_tune_epochs, save_path=None):
    scaler = torch.amp.GradScaler()
    model.train()

    for epoch in range(train_epochs + fine_tune_epochs):
        epoch_loss = 0.0

        # Gradual unfreezing of pre-trained models
        if epoch < 5:
            for param in model.swin_model.parameters():
                param.requires_grad = False
            for param in model.xlmr_model.parameters():
                param.requires_grad = False
        else:
            for param in model.swin_model.parameters():
                param.requires_grad = True
            for param in model.xlmr_model.parameters():
                param.requires_grad = True

        loader = train_loader if epoch < train_epochs else fine_tune_loader
        phase = "Training" if epoch < train_epochs else "Fine-Tuning"

        for images, texts, attention_masks, labels in loader:
            images, texts, attention_masks, labels = (
                images.to(device),
                texts.to(device),
                attention_masks.to(device),
                labels.to(device),
            )
            optimizer.zero_grad()

            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images, texts, attention_masks).squeeze()
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            epoch_loss += loss.item()

        print(f"{phase} Epoch [{epoch + 1}/{train_epochs + fine_tune_epochs}], Loss: {epoch_loss / len(loader):.4f}")

    if save_path:
        save_model(model, save_path)

# Save and Load Functions
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))
        model.eval()
        print(f"Model loaded from {path}")
    else:
        raise FileNotFoundError(f"Model file not found at {path}")

# Evaluation Function
def evaluate_model(model, dataloader, device):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, texts, attention_masks, labels in dataloader:
            images, texts, attention_masks, labels = (
                images.to(device),
                texts.to(device),
                attention_masks.to(device),
                labels.to(device),
            )
            outputs = model(images, texts, attention_masks).squeeze()
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
    return accuracy, precision, recall, f1

# Paths
train_file = "/home/kalyanreddy/Datasets/English/Train.jsonl"
fine_tune_file = "/home/kalyanreddy/Datasets/Finetune.json"
test_file = "/home/kalyanreddy/Datasets/MultiTest.json"
model_save_path = "multimodal_clip_model_xlmr.pth"

# DataLoader parameters
batch_size = 64

# Load datasets
train_data = load_dataset(train_file, is_jsonl=True)
fine_tune_data = load_dataset(fine_tune_file, is_jsonl=False)
test_data = load_dataset(test_file, is_jsonl=False)

train_loader = DataLoader(MultimodalDataset(train_data, image_transform, xlmr_tokenizer), batch_size=batch_size, shuffle=True)
fine_tune_loader = DataLoader(MultimodalDataset(fine_tune_data, image_transform, xlmr_tokenizer), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(MultimodalDataset(test_data, image_transform, xlmr_tokenizer), batch_size=batch_size, shuffle=False)

# Initialize model
multimodal_model = MultimodalClassifier(
    swin_model_name="swin_base_patch4_window7_224",
    xlmr_model_name="xlm-roberta-base",
    fusion_dim=512
).to(device)

# Loss, Optimizer, and Scheduler
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(
    [{'params': multimodal_model.clip_fusion.parameters(), 'lr': 1e-4},
     {'params': multimodal_model.swin_model.parameters(), 'lr': 1e-5},
     {'params': multimodal_model.xlmr_model.parameters(), 'lr': 1e-5}],
    weight_decay=1e-2
)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)





/home/kalyanreddy/miniconda3/envs/Clip_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Train and Fine-Tune Model
train_and_fine_tune_model(
    multimodal_model,
    train_loader,
    fine_tune_loader,
    criterion,
    optimizer,
    scheduler,
    device,
    train_epochs=20,
    fine_tune_epochs=10,
    save_path=model_save_path
)

Training Epoch [1/30], Loss: 0.6606
Training Epoch [2/30], Loss: 0.6543
Training Epoch [3/30], Loss: 0.6464
Training Epoch [4/30], Loss: 0.6417
Training Epoch [5/30], Loss: 0.6388
Training Epoch [6/30], Loss: 0.6243
Training Epoch [7/30], Loss: 0.5766
Training Epoch [8/30], Loss: 0.5461
Training Epoch [9/30], Loss: 0.5238
Training Epoch [10/30], Loss: 0.5051
Training Epoch [11/30], Loss: 0.4943
Training Epoch [12/30], Loss: 0.4672
Training Epoch [13/30], Loss: 0.4453
Training Epoch [14/30], Loss: 0.4260
Training Epoch [15/30], Loss: 0.4056
Training Epoch [16/30], Loss: 0.3901
Training Epoch [17/30], Loss: 0.3636
Training Epoch [18/30], Loss: 0.3439
Training Epoch [19/30], Loss: 0.3300
Training Epoch [20/30], Loss: 0.3196
Fine-Tuning Epoch [21/30], Loss: 0.8641
Fine-Tuning Epoch [22/30], Loss: 0.7235
Fine-Tuning Epoch [23/30], Loss: 0.6324
Fine-Tuning Epoch [24/30], Loss: 0.6152
Fine-Tuning Epoch [25/30], Loss: 0.5735
Fine-Tuning Epoch [26/30], Loss: 0.5285
Fine-Tuning Epoch [27/30], Lo

In [7]:
# Evaluate Model
evaluate_model(multimodal_model, test_loader, device)

Accuracy: 69.32%
Precision: 0.69, Recall: 0.71, F1-Score: 0.70


(0.6931818181818182,
 0.6883116883116883,
 0.7076101468624834,
 0.6978275181040158)